In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI
from Exercise03AI import Exercise03AI

# Aufgabe 05: Minimax mit Alpha-Beta-Pruning

Dieses Notebook implementiert den Minimax-Algorithmus mit Alpha-Beta-Pruning (ohne Memoisierung). Hierzu wird die `minimax`-Funktion verändert (überschrieben).

In [ ]:
import chess_custom as chess

class Exercise05AI(Exercise03AI):
    """Chooses middle game moves using minimax algorithm and alpha-beta-pruning"""
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)

## Minimax (aktualisiert)

Die folgende Implementierung des Minimax-Algorithmus ist zu großen Teilen mit der von `Exercise02AI` identisch. Als Neuerung wurde Alpha-Beta-Pruning hinzugefügt. Im Folgenden werden nur die Änderungen betrachtet:

1. Die `minimax`-Funktion bekommt zwei zusätzliche Parameter $alpha$ und $beta$ welche mithilfe von Standardwerten beim ersten Aufruf auf das positive bzw. negative Limit gesetzt werden.
2. Wenn der weiße Spieler am Zug ist (Maximierung) wird `maxEvaluation` auf den Wert $alpha$ gesetzt. Wie bisher wird nun jeder Zug evaluiert. Hierbei wird das Maximum der bisherigen Evaluationen jeweils als neue untere Grenze $alpha$ eingesetzt. Falls eine Evaluierung größer oder gleich $beta$ ist, wird die Suche abgebrochen (pruning) und diese Evaluierung zurückgegeben.
2. Falls der schwarze Spieler am Zug ist (Minimierung) wird `minEvaluation` auf den Wert $beta$ gesetzt. Wie bisher wird nun jeder Zug evaluiert. Hierbei wird das Minimum der bisherigen Evaluationen jeweils als neue obere Grenze $beta$ eingesetzt. Falls eine Evaluierung kleiner oder gleich $alpha$ ist, wird die Suche abgebrochen (pruning) und diese Evaluierung zurückgegeben.

Für einen Aufruf mit Weiß am Zug gilt also: 
$$return\_value = \begin{cases}
alpha \quad \texttt{falls} \quad evaluation < alpha \quad \texttt{für alle} \quad move \in board.legal\_moves,\\
\ge beta \quad \texttt{falls} \quad move \in board.legal\_moves \quad \texttt{mit} \quad evaluation \ge beta \quad \texttt{existiert,}\\
evaluation \quad \texttt{sonst}.
\end{cases}$$
Für einen Aufruf mit Schwarz am Zug gilt: 
$$return\_value = \begin{cases}
beta \quad \texttt{falls} \quad evaluation > beta \quad \texttt{für alle} \quad move \in board.legal\_moves,\\
\le alpha \quad \texttt{falls} \quad move \in board.legal\_moves \quad \texttt{mit} \quad evaluation \le alpha \quad \texttt{existiert,}\\
evaluation \quad \texttt{sonst}.
\end{cases}$$
Somit ist in beiden Fällen die Spezifikation für das Alpha-Beta-Pruning erfüllt und das Programm liefert dieselbe Auswertung wie die Implementierung in `Exercise02AI`.

In [ ]:
from typing import Any

class Exercise05AI(Exercise05AI): # type: ignore
    
    def minimax(self, board: chess.Board, depth: int, current_evaluation: int,
                alpha: int = -Exercise05AI.LIMIT, beta: int = Exercise05AI.LIMIT) -> tuple[int, chess.Move]:
        """Searches the best value with a given depth using the minimax algorithm"""
        best_move = None
        
        if (is_checkmate := board.is_checkmate()) and not board.turn:
            # White has won the game
            evaluation = self.LIMIT - (self.DEPTH - depth)
            return evaluation, None
        elif is_checkmate and board.turn:
            # Black has won the game
            evaluation = -self.LIMIT + (self.DEPTH - depth)
            return evaluation, None
        elif board.is_insufficient_material() or not board.legal_moves or board.is_fifty_moves() or board.is_repetition(5):
            # Game is a draw
            return 0, None

        # Recursion abort case
        if depth == 0:
            return current_evaluation, None

        # White to play (positive numbers are good)
        if board.turn:
            maxEvaluation = alpha
            for move in board.legal_moves:
                board.push(move)
                evaluation, _ = self.minimax(board, depth - 1, self.evaluate(board, current_evaluation), maxEvaluation, beta)
                board.pop()
                if evaluation >= beta:
                    return evaluation, move
                if depth == self.DEPTH and evaluation > maxEvaluation:
                    best_move = move
                maxEvaluation = max(maxEvaluation, evaluation)
            return maxEvaluation, best_move

        # Black to play (negative numbers are good)
        else:
            minEvaluation = beta
            for move in board.legal_moves:
                board.push(move)
                evaluation, _ = self.minimax(board, depth - 1, self.evaluate(board, current_evaluation), alpha, minEvaluation)
                board.pop()
                if evaluation <= alpha:
                    return evaluation, move
                if depth == self.DEPTH and evaluation < minEvaluation:
                    best_move = move
                minEvaluation = min(minEvaluation, evaluation)
            return minEvaluation, best_move

## Debugging Bereich

Die folgenden Zellen enthalten Code zum Testen der oben implementierten Funktionen.

In [ ]:
%xmode Plain

In [ ]:
#debug
from Exercise03AI import Exercise03AI

In [ ]:
#debug
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 w - - 0 36")
board.push(chess.Move.from_uci("h2h1"))
board

In [ ]:
%%time
DEPTH = 3
player3 = Exercise03AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player3.get_next_middle_game_move(board)
print(move)

In [ ]:
%%time
DEPTH = 3
player5 = Exercise05AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player5.get_next_middle_game_move(board)
print(move)

In [ ]:
#debug
from IPython.display import clear_output, display
player5 = Exercise05AI("Testplayer", 3)
player3 = Exercise03AI("Testplayer", 3)
move = player3.random.choice(list(board.legal_moves))
board.push(move)
for _ in range(10):
    move_1, eval_1 = player3.get_next_middle_game_move(board)
    player3.last_evaluation = None
    print(move_1, eval_1, player3.last_evaluation)
    move_2, eval_2 = player5.get_next_middle_game_move(board)
    player5.last_evaluation = None
    print(move_2, eval_2, player5.last_evaluation)
    move = player3.random.choice(list(board.legal_moves))
    board.push(move)

In [ ]:
%%time
DEPTH = 3
board = chess.Board()
#board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
for _ in range(10):
    board.push(sorted(board.legal_moves, key=lambda move: move.uci())[0])
    move = player.get_next_middle_game_move(board)
    print(move)

In [ ]:
#debug
from os.path import join

with open(join("..", "games", "2022-01-15_20-18-57-907777.pgn")) as pgn:
    first_game = chess.pgn.read_game(pgn)

# Iterate through all moves and play them on a board.
board = first_game.board()
for move in first_game.mainline_moves():
    board.push(move)
for i in range(220):
    board.pop()

In [ ]:
%%time
DEPTH = 2
#board = chess.Board()
#board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise04AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.get_next_middle_game_move(board)
print(move)
player.cache_hits

In [ ]:
#debug
board = chess.Board("7B/pbpk4/1p6/2n1pp2/6p1/4P3/P1P2PNP/RNq1K1R1 w - - 0 1")
player = Exercise04AI("Testplayer", 3)
print(player.evaluate(board))
board